In [17]:
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
# Import Linear Regression and a regularized regression function
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
# Finally, import function to make a machine learning pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import ExtraTreesClassifier


## Load Data

In [6]:
X = pd.read_csv('X_train.csv')
Y = pd.read_csv('Y_train.csv')
X_test = pd.read_csv('X_test.csv')

print("Training Data:{}".format(X.shape))
print("Test Data:{}".format(X_test.shape))

Y.head(2)


Training Data:(1212, 888)
Test Data:(776, 888)


,id,y
0,0.0,75.0
1,1.0,76.0


In [7]:
Y['y'].describe()

count    1212.000000
mean       69.763201
std         9.941656
min        42.000000
25%        64.000000
50%        70.000000
75%        76.000000
max        96.000000
Name: y, dtype: float64

In [8]:
X.describe()

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x877,x878,x879,x880,x881,x882,x883,x884,x885,x886
count,1212.000000,1122.000000,1.140000e+03,1.132000e+03,1123.000000,1133.000000,1126.000000,1141.000000,1122.000000,1122.000000,...,1.127000e+03,1.127000e+03,1.132000e+03,1127.000000,1.125000e+03,1127.000000,1148.000000,1126.000000,1123.000000,1111.000000
mean,605.500000,7300.504957,1.003125e+06,1.051614e+06,1049.844772,105.047681,203511.156265,1050.735880,341958.172245,104916.372111,...,3.825322e+11,1.003145e+06,-5.025956e+05,1001.891614,9.995905e+05,3732.365716,100.659348,1617.956555,10505.966555,65052.578569
std,350.018571,1379.891266,1.001817e+05,2.818085e+04,28.475255,2.823009,29841.633207,28.623527,58820.438523,2755.013692,...,4.347959e+11,9.594847e+04,8.608874e+04,100.410174,9.680491e+04,725.532171,9.336065,401.791865,290.648021,0.029221
min,0.000000,1030.502715,6.716345e+05,1.000037e+06,1000.062471,100.033879,63202.600024,1000.134779,92365.078214,100016.602565,...,-5.083882e+11,7.186635e+05,-1.110029e+06,643.042857,6.895354e+05,451.131089,65.692019,458.289896,10001.346875,65052.528022
25%,302.750000,6496.988432,9.409699e+05,1.028118e+06,1025.913567,102.724769,186609.583069,1026.464126,309182.739540,102687.100342,...,1.578603e+11,9.385117e+05,-5.475966e+05,933.591537,9.348641e+05,3297.203036,94.515998,1360.553119,10249.981685,65052.553207
50%,605.500000,7381.752216,1.003238e+06,1.052406e+06,1050.174694,105.023063,201709.971057,1051.399190,337308.178918,104861.600927,...,2.758197e+11,1.001974e+06,-4.965350e+05,1001.295903,9.989761e+05,3768.931107,100.672131,1604.528424,10505.538263,65052.579678
75%,908.250000,8153.767104,1.070372e+06,1.075329e+06,1074.864998,107.391464,220981.402036,1075.166305,371797.754187,107160.482832,...,4.867297e+11,1.063238e+06,-4.560057e+05,1069.927335,1.064618e+06,4179.602230,106.809319,1861.784028,10763.810688,65052.603107
max,1211.000000,13055.814408,1.316548e+06,1.099990e+06,1099.845375,110.048177,370398.522988,1099.997865,784817.830992,109991.914244,...,7.405700e+12,1.308895e+06,-1.400403e+05,1323.073354,1.276136e+06,6781.164024,126.678078,3745.022165,10999.908941,65052.627907


## Preprocessing

### Replace missing values: Interpolation

In [9]:
print("Missing entries across all features: ",X.isnull().sum().sum())
X[X.x0.isnull()].head(5)

Missing entries across all features:  77121


,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x877,x878,x879,x880,x881,x882,x883,x884,x885,x886
8,8.0,NaN,1.136943e+06,1.047700e+06,1095.071770,104.280469,199679.789379,1075.166305,308952.165532,NaN,...,6.546477e+10,NaN,-454744.713147,845.220975,1.124313e+06,4242.573171,89.842734,983.020034,10064.518546,65052.596763
15,15.0,NaN,1.082081e+06,1.053115e+06,1050.318884,107.479167,214460.503578,1012.636406,325226.367066,NaN,...,NaN,8.897315e+05,NaN,1050.097266,1.126804e+06,3840.596957,100.609133,1777.062732,10722.666402,65052.534048
17,17.0,NaN,1.139222e+06,1.089087e+06,1043.829138,NaN,NaN,1091.372597,311692.655155,106852.971402,...,NaN,8.396675e+05,-457813.748540,914.165982,1.215597e+06,3470.879134,94.259174,1512.026531,10065.973104,NaN
49,49.0,NaN,1.141092e+06,1.036322e+06,1097.841819,100.595510,192700.231292,1012.064796,349769.421213,109055.678089,...,6.351533e+11,1.072613e+06,-509825.273218,969.096337,8.303299e+05,4486.555772,92.507889,2008.006951,10836.452970,65052.555294
60,60.0,NaN,9.635644e+05,1.072941e+06,1081.119503,101.504822,221232.605200,1013.414998,373612.773713,106301.596339,...,3.062288e+11,7.953379e+05,-548945.945151,919.380354,9.609103e+05,3854.713301,107.829104,1638.064765,10877.091027,65052.530389


In [10]:
X.x0.head(5)

0    7077.537454
1    4406.423818
2    4460.878690
3    7152.388016
4    6531.930242
Name: x0, dtype: float64

In [11]:
X.x0 =  X.x0.interpolate(method='nearest')



In [12]:
# X_train.iloc[8:60]
# X_train[X_train.x0.isnull()].head(5)

In [13]:
X = X.interpolate(method = 'nearest').ffill().bfill()

X.isnull().sum().sum()

0

### Data normalization

In [14]:
X_normalized = preprocessing.normalize(X)
X_standardized = preprocessing.scale(X)


/anaconda/envs/py3k/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


### Training / Validation Spilt

In [40]:
X_train, X_val, Y_train, Y_val = train_test_split(X_normalized, Y, test_size=0.2, random_state=42)
print("X_train: {}".format(X_train.shape[0]))
print("X_test: {}".format(X_test.shape[0]))



X_train: 969
X_test: 243


## Feature Reduction / Extraction

In [41]:
# feature extraction
model = ExtraTreesClassifier()
model.fit(X_train, Y_train)
feature_importance = model.feature_importances_

In [42]:
# feature_importance.find(feature_importance>0)

feature_max_importance = np.argwhere(feature_importance > 0)
feature_max_importance

array([[332],
       [401],
       [610],
       [878]])

I will select the following features

In [43]:
X_train_selected = X_train[:,feature_max_importance]
X_val_selected = X_val[:,feature_max_importance]

X_train_selected = X_train_selected.reshape(X_train_selected.shape[0],X_train_selected.shape[1])
X_train_selected.shape

X_val_selected = X_val_selected.reshape(X_val_selected.shape[0],X_val_selected.shape[1])
X_val_selected.shape

(243, 4)

## Algorithm

In [44]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train_selected, Y_train)

# Make predictions using the testing set
Y_pred = regr.predict(X_val_selected)


print('R2 score: %.2f' % r2_score(Y_val, Y_pred))


R2 score: 0.00


In [45]:
# Alpha (regularization strength) of LASSO regression
# lasso_eps = 0.0001
# lasso_nalpha=20
# lasso_iter=5000
# # Min and max degree of polynomials features to consider
# degree_min = 2
# degree_max = 8
# Test/train split
# Make a pipeline model with polynomial transformation and LASSO regression with cross-validation, run it for increasing degree of polynomial (complexity of the model)
# for degree in range(degree_min,degree_max+1):
#     model = make_pipeline(PolynomialFeatures(degree, interaction_only=False), LassoCV(eps=lasso_eps,n_alphas=lasso_nalpha,max_iter=lasso_iter,
# normalize=True,cv=5))
#     model.fit(X_train,Y_train)
#     Y_pred = np.array(model.predict(X_test))
    
#     print('R2 score: %.2f' % r2_score(Y_test, Y_pred))

    